## Aqui fazemos a primeira etapa do code, onde é feito o processamento dos dados baixados da ação correspondente, direto do site da Fundamentus. São explicados todos os passos do processamento. 

In [1]:
import numpy as np
import pandas as pd
import pandas_datareader.data as web
from datetime import datetime
import yfinance as yf
from fancyimpute import KNN


pd.set_option('display.width', 1000)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

Using TensorFlow backend.


# DADOS FUNDAMENTALISTAS - FUNDAMENTUS

Ao fazer o download do site da Fundamentus, os dados vem em formato excel, então pegamos e convertemos facilmente para cvs.

Aqui trazemos o csv da ação para o notebook. Ele necessita do ".T" para que ele faça uma transposição, pois os atributos estavam nas linhas e as amostras nas colunas, assim invertemos isso, ficando no formato convencional utilizado.

Após, dropamos uma coluna inútil.

A coluna que mostra as datas, que estão separadas por trimestre, fica com um título NaN e está na primeira posição das colunas, assim precisamos atribuir o devido nome a coluna.

Assim, criamos uma coluna chamada "Data Trimeste", ao que lhe é atribuida as datas, e na sequência deletamos a coluna com o títlo NaN, através do iloc que engloba apenas colunas que tenham título "notnull"(naõ nulos).


A coluna "Data Trimestre" fica posicionada como última coluna do dataset, para ficar mais claro, apagamos, colocamos em uma variavel auxiliar, e a inserimos na primeira posição das colunas.

In [2]:
########################## COMEÇA O CÓDIGO QUE EXTRAI OS DADOS FUNDAMENTALISTAS ##########################

fundamental_data = pd.read_csv("lame4.csv",index_col=0, header=None).T
fundamental_data.drop(columns = 'XLSWrite 1.34 Copyright(c) 1999,2000 Axolot Data', inplace = True)


fundamental_data['Data Trimestre'] = fundamental_data.iloc[0:,0]
fundamental_data = fundamental_data.iloc[:, fundamental_data.columns.notnull()]

data = fundamental_data['Data Trimestre']
fundamental_data.drop(labels=['Data Trimestre'], axis=1,inplace = True)
fundamental_data.insert(0, 'Data Trimestre', data)


fundamental_data.head()

,Data Trimestre,Ativo Total,Ativo Circulante,Caixa e Equivalentes de Caixa AC,Aplicações Financeiras AC,Contas a Receber AC,Estoques AC,Ativos Biológicos AC,Tributos a Recuperar AC,Despesas Antecipadas AC,Outros Ativos Circulantes AC,Ativo Realizavel Longo Prazo,Aplicações Financeiras Avaliadas a Valor Justo ACLP,Aplicações Financeiras Avaliadas ao Custo Amortizado ACLP,Contas a Receber ACLP,Estoques ACLP,Ativos Biológicos ACLP,Tributos Diferidos ACLP,Despesas Antecipadas ACLP,Créditos com Partes Relacionadas ACLP,Outros Ativos Não Circulantes ACLP,Investimentos,Imobilizado,Intangível,Diferido,Passivo Total,Passivo Circulante,Obrigações Sociais e Trabalhistas PC,Fornecedores PC,Obrigações Fiscais PC,Empréstimos e Financiamentos PC,Passivos com Partes Relacionadas PC,Dividendos e JCP a Pagar PC,Outros PC,Provisões PC,Passivos sobre Ativos Não-Correntes a Venda e Descontinuados PC,Passivo Não Circulante,Empréstimos e Financiamentos PNC,Passivos com Partes Relacionadas PNC,Outros PNC,Tributos Diferidos PNC,Adiantamento para Futuro Aumento Capital PNC,Provisões PNC,Passivos sobre Ativos Não-Correntes a Venda e Descontinuados PNC,Lucros e Receitas a Apropriar PNC,Participação dos Acionistas Não Controladores,Patrimônio Líquido,Capital Social Realizado,Reservas de Capital,Reservas de Reavaliação,Reservas de Lucros,Lucros/Prejuízos Acumulados,Ajustes de Avaliação Patrimonial,Ajustes Acumulados de Conversão,Outros Resultados Abrangentes,Adiantamento para Futuro Aumento Capital
1,30/06/2019,30.466.716,17.170.224,6.573.441,3.947.952,1.576.149,3.123.935,0,886.679,83.323,978.744,3.729.933,61.476,0,0,0,0,0,1.370.409,0,46.702,0,5.720.875,3.845.686,0,30.466.716,7.549.925,156.515,3.520.434,178.062,3.168.946,0,22.600,921,502.447,0,16.876.870,16.665.291,0,8.251,0,0,203.328,0,0,1.254.018,4.785.904,3.957.961,115.306,0,623.837,59.240,29.560,0,0,0
2,31/03/2019,32.632.261,19.639.284,7.725.360,4.616.139,1.695.657,3.795.014,0,808.076,79.648,919.390,3.701.606,41.417,0,0,0,0,0,1.361.352,0,51.464,0,5.495.443,3.795.926,0,32.632.261,8.874.362,145.691,4.698.474,161.449,3.230.820,0,126.215,1.229,510.484,0,17.794.312,17.577.052,0,8.160,0,0,209.101,0,0,1.301.020,4.662.565,3.957.961,102.809,0,623.837,-53.484,31.442,0,0,0
3,31/12/2018,28.304.656,17.513.619,6.813.846,3.295.849,1.870.081,3.506.678,0,906.836,59.036,1.061.295,3.380.094,0,0,0,0,0,0,1.197.780,0,51.008,0,3.647.720,3.763.221,0,28.304.656,8.402.748,152.235,4.973.577,325.947,2.290.854,0,126.215,1.534,532.386,0,13.615.783,13.389.983,0,8.039,0,0,217.761,0,0,1.361.528,4.924.597,3.957.961,90.311,0,845.540,0,30.785,0,0,0
4,30/09/2018,26.876.613,16.403.945,5.982.687,3.229.278,1.335.874,3.639.626,0,967.622,76.478,1.172.380,3.198.663,0,0,0,0,0,0,1.155.231,0,50.626,0,3.538.550,3.735.453,0,26.876.613,6.853.386,159.582,3.641.575,236.674,2.350.935,0,0,14.614,450.006,0,13.860.291,13.649.758,0,8.079,0,0,202.454,0,0,1.388.215,4.774.719,3.957.961,78.381,0,581.485,107.688,49.204,0,0,0
5,30/06/2018,26.015.085,15.810.592,5.201.874,4.158.143,1.126.535,3.207.755,0,843.403,73.311,1.199.571,3.052.373,35.840,0,0,0,0,0,1.104.241,0,47.900,0,3.429.345,3.722.775,0,26.015.085,6.475.659,145.617,2.814.037,208.490,2.933.797,0,0,19.566,354.152,0,13.427.922,13.195.491,0,8.364,0,0,224.067,0,0,1.402.201,4.709.303,3.926.518,95.002,0,597.146,46.294,44.343,0,0,0


Aqui é o processo de manipulação das datas, muito importante na execução do code.

Primeiro utilizamos pd.to_datetime, função própria do Pandas para que ele entenda o formato e de que se trata de informações sobre data.

Subdividimos em colunas distintas, informações sobre "Dia", "Mês" e "Ano", logo a coluna "Data Trimestre" já não nos era mais útil.

Assim, novamente colocamos as três novas colunas em váriaveis auxiliares, para posiciona-las nas primeiras colunas do dataset.

In [3]:

fundamental_data['Data Trimestre'] = pd.to_datetime(fundamental_data['Data Trimestre'].str.strip(), format='%d/%m/%Y')


fundamental_data["Day"] = fundamental_data['Data Trimestre'].map(lambda x: x.day)
fundamental_data["Month"] = fundamental_data['Data Trimestre'].map(lambda x: x.month)
fundamental_data["Year"] = fundamental_data['Data Trimestre'].map(lambda x: x.year)

fundamental_data.drop(columns = 'Data Trimestre', inplace = True)

year = fundamental_data['Year']
fundamental_data.drop(labels=['Year'], axis=1,inplace = True)
fundamental_data.insert(0, 'Year', year)

month = fundamental_data['Month']
fundamental_data.drop(labels=['Month'], axis=1,inplace = True)
fundamental_data.insert(1, 'Month', month)

day = fundamental_data['Day']
fundamental_data.drop(labels=['Day'], axis=1,inplace = True)
fundamental_data.insert(2, 'Day', day)


fundamental_data.head()

,Year,Month,Day,Ativo Total,Ativo Circulante,Caixa e Equivalentes de Caixa AC,Aplicações Financeiras AC,Contas a Receber AC,Estoques AC,Ativos Biológicos AC,Tributos a Recuperar AC,Despesas Antecipadas AC,Outros Ativos Circulantes AC,Ativo Realizavel Longo Prazo,Aplicações Financeiras Avaliadas a Valor Justo ACLP,Aplicações Financeiras Avaliadas ao Custo Amortizado ACLP,Contas a Receber ACLP,Estoques ACLP,Ativos Biológicos ACLP,Tributos Diferidos ACLP,Despesas Antecipadas ACLP,Créditos com Partes Relacionadas ACLP,Outros Ativos Não Circulantes ACLP,Investimentos,Imobilizado,Intangível,Diferido,Passivo Total,Passivo Circulante,Obrigações Sociais e Trabalhistas PC,Fornecedores PC,Obrigações Fiscais PC,Empréstimos e Financiamentos PC,Passivos com Partes Relacionadas PC,Dividendos e JCP a Pagar PC,Outros PC,Provisões PC,Passivos sobre Ativos Não-Correntes a Venda e Descontinuados PC,Passivo Não Circulante,Empréstimos e Financiamentos PNC,Passivos com Partes Relacionadas PNC,Outros PNC,Tributos Diferidos PNC,Adiantamento para Futuro Aumento Capital PNC,Provisões PNC,Passivos sobre Ativos Não-Correntes a Venda e Descontinuados PNC,Lucros e Receitas a Apropriar PNC,Participação dos Acionistas Não Controladores,Patrimônio Líquido,Capital Social Realizado,Reservas de Capital,Reservas de Reavaliação,Reservas de Lucros,Lucros/Prejuízos Acumulados,Ajustes de Avaliação Patrimonial,Ajustes Acumulados de Conversão,Outros Resultados Abrangentes,Adiantamento para Futuro Aumento Capital
1,2019,6,30,30.466.716,17.170.224,6.573.441,3.947.952,1.576.149,3.123.935,0,886.679,83.323,978.744,3.729.933,61.476,0,0,0,0,0,1.370.409,0,46.702,0,5.720.875,3.845.686,0,30.466.716,7.549.925,156.515,3.520.434,178.062,3.168.946,0,22.600,921,502.447,0,16.876.870,16.665.291,0,8.251,0,0,203.328,0,0,1.254.018,4.785.904,3.957.961,115.306,0,623.837,59.240,29.560,0,0,0
2,2019,3,31,32.632.261,19.639.284,7.725.360,4.616.139,1.695.657,3.795.014,0,808.076,79.648,919.390,3.701.606,41.417,0,0,0,0,0,1.361.352,0,51.464,0,5.495.443,3.795.926,0,32.632.261,8.874.362,145.691,4.698.474,161.449,3.230.820,0,126.215,1.229,510.484,0,17.794.312,17.577.052,0,8.160,0,0,209.101,0,0,1.301.020,4.662.565,3.957.961,102.809,0,623.837,-53.484,31.442,0,0,0
3,2018,12,31,28.304.656,17.513.619,6.813.846,3.295.849,1.870.081,3.506.678,0,906.836,59.036,1.061.295,3.380.094,0,0,0,0,0,0,1.197.780,0,51.008,0,3.647.720,3.763.221,0,28.304.656,8.402.748,152.235,4.973.577,325.947,2.290.854,0,126.215,1.534,532.386,0,13.615.783,13.389.983,0,8.039,0,0,217.761,0,0,1.361.528,4.924.597,3.957.961,90.311,0,845.540,0,30.785,0,0,0
4,2018,9,30,26.876.613,16.403.945,5.982.687,3.229.278,1.335.874,3.639.626,0,967.622,76.478,1.172.380,3.198.663,0,0,0,0,0,0,1.155.231,0,50.626,0,3.538.550,3.735.453,0,26.876.613,6.853.386,159.582,3.641.575,236.674,2.350.935,0,0,14.614,450.006,0,13.860.291,13.649.758,0,8.079,0,0,202.454,0,0,1.388.215,4.774.719,3.957.961,78.381,0,581.485,107.688,49.204,0,0,0
5,2018,6,30,26.015.085,15.810.592,5.201.874,4.158.143,1.126.535,3.207.755,0,843.403,73.311,1.199.571,3.052.373,35.840,0,0,0,0,0,1.104.241,0,47.900,0,3.429.345,3.722.775,0,26.015.085,6.475.659,145.617,2.814.037,208.490,2.933.797,0,0,19.566,354.152,0,13.427.922,13.195.491,0,8.364,0,0,224.067,0,0,1.402.201,4.709.303,3.926.518,95.002,0,597.146,46.294,44.343,0,0,0


Essa é uma parte bem crítica, resetamos o índice(para que ele fique com a contagem correta de 0 até o total de linhas). E você deve estar se perguntando: por quê pegar o dataset a partir da 4 posição ?

Esse dataset está em formato descrescente com base nas datas, então, la no final do projeto, quando formos comparar os algortimos de classificação com a série temporal, para validação temos que comparar com dados que já ocorreram, par ter uma noção do desempenho.

Lembrando que se pegassemos de 0:, nesse caso, ele ia predizer dados a partir do 3T de 2018 até o 3T de 2019, sendo que o TCC estava sendo feito nessa época, então não tinhamos os dados reais necessarios para comparação, então por fins logisticos, reduzimos a ánalise um ano antes, para ao final do trabalho, quando compararmos os fundamentus e os algortimos de classificação com as séries temporais e a LSTM tenhamos tranquilidade para comparar com dados que ocorreram de fato para se ter uma boa base para inferir desempenho e tirar conclusões.

In [4]:
fundamental_data.reset_index(drop=True, inplace=True)
fundamental_data = fundamental_data[0:42]
fundamental_data.reset_index(drop=True, inplace=True)

fundamental_data.head()

,Year,Month,Day,Ativo Total,Ativo Circulante,Caixa e Equivalentes de Caixa AC,Aplicações Financeiras AC,Contas a Receber AC,Estoques AC,Ativos Biológicos AC,Tributos a Recuperar AC,Despesas Antecipadas AC,Outros Ativos Circulantes AC,Ativo Realizavel Longo Prazo,Aplicações Financeiras Avaliadas a Valor Justo ACLP,Aplicações Financeiras Avaliadas ao Custo Amortizado ACLP,Contas a Receber ACLP,Estoques ACLP,Ativos Biológicos ACLP,Tributos Diferidos ACLP,Despesas Antecipadas ACLP,Créditos com Partes Relacionadas ACLP,Outros Ativos Não Circulantes ACLP,Investimentos,Imobilizado,Intangível,Diferido,Passivo Total,Passivo Circulante,Obrigações Sociais e Trabalhistas PC,Fornecedores PC,Obrigações Fiscais PC,Empréstimos e Financiamentos PC,Passivos com Partes Relacionadas PC,Dividendos e JCP a Pagar PC,Outros PC,Provisões PC,Passivos sobre Ativos Não-Correntes a Venda e Descontinuados PC,Passivo Não Circulante,Empréstimos e Financiamentos PNC,Passivos com Partes Relacionadas PNC,Outros PNC,Tributos Diferidos PNC,Adiantamento para Futuro Aumento Capital PNC,Provisões PNC,Passivos sobre Ativos Não-Correntes a Venda e Descontinuados PNC,Lucros e Receitas a Apropriar PNC,Participação dos Acionistas Não Controladores,Patrimônio Líquido,Capital Social Realizado,Reservas de Capital,Reservas de Reavaliação,Reservas de Lucros,Lucros/Prejuízos Acumulados,Ajustes de Avaliação Patrimonial,Ajustes Acumulados de Conversão,Outros Resultados Abrangentes,Adiantamento para Futuro Aumento Capital
0,2019,6,30,30.466.716,17.170.224,6.573.441,3.947.952,1.576.149,3.123.935,0,886.679,83.323,978.744,3.729.933,61.476,0,0,0,0,0,1.370.409,0,46.702,0,5.720.875,3.845.686,0,30.466.716,7.549.925,156.515,3.520.434,178.062,3.168.946,0,22.600,921,502.447,0,16.876.870,16.665.291,0,8.251,0,0,203.328,0,0,1.254.018,4.785.904,3.957.961,115.306,0,623.837,59.240,29.560,0,0,0
1,2019,3,31,32.632.261,19.639.284,7.725.360,4.616.139,1.695.657,3.795.014,0,808.076,79.648,919.390,3.701.606,41.417,0,0,0,0,0,1.361.352,0,51.464,0,5.495.443,3.795.926,0,32.632.261,8.874.362,145.691,4.698.474,161.449,3.230.820,0,126.215,1.229,510.484,0,17.794.312,17.577.052,0,8.160,0,0,209.101,0,0,1.301.020,4.662.565,3.957.961,102.809,0,623.837,-53.484,31.442,0,0,0
2,2018,12,31,28.304.656,17.513.619,6.813.846,3.295.849,1.870.081,3.506.678,0,906.836,59.036,1.061.295,3.380.094,0,0,0,0,0,0,1.197.780,0,51.008,0,3.647.720,3.763.221,0,28.304.656,8.402.748,152.235,4.973.577,325.947,2.290.854,0,126.215,1.534,532.386,0,13.615.783,13.389.983,0,8.039,0,0,217.761,0,0,1.361.528,4.924.597,3.957.961,90.311,0,845.540,0,30.785,0,0,0
3,2018,9,30,26.876.613,16.403.945,5.982.687,3.229.278,1.335.874,3.639.626,0,967.622,76.478,1.172.380,3.198.663,0,0,0,0,0,0,1.155.231,0,50.626,0,3.538.550,3.735.453,0,26.876.613,6.853.386,159.582,3.641.575,236.674,2.350.935,0,0,14.614,450.006,0,13.860.291,13.649.758,0,8.079,0,0,202.454,0,0,1.388.215,4.774.719,3.957.961,78.381,0,581.485,107.688,49.204,0,0,0
4,2018,6,30,26.015.085,15.810.592,5.201.874,4.158.143,1.126.535,3.207.755,0,843.403,73.311,1.199.571,3.052.373,35.840,0,0,0,0,0,1.104.241,0,47.900,0,3.429.345,3.722.775,0,26.015.085,6.475.659,145.617,2.814.037,208.490,2.933.797,0,0,19.566,354.152,0,13.427.922,13.195.491,0,8.364,0,0,224.067,0,0,1.402.201,4.709.303,3.926.518,95.002,0,597.146,46.294,44.343,0,0,0


Para fins didáticos, criei uma coluna "Quarter" que indica qual trimestre corresponde aquela linha, e a posicionei na primeira posição das colunas.

In [5]:
fundamental_data['Quarter'] = '2Q2019','1Q2019', '4Q2018', '3Q2018', '2Q2018', '1Q2018', '4Q2017', '3Q2017', '2Q2017', '1Q2017', '4Q2016', '3Q2016', '2Q2016', '1Q2016', '4Q2015', '3Q2015', '2Q2015', '1Q2015','4Q2014', '3Q2014', '2Q2014', '1Q2014', '4Q2013', '3Q2013', '2Q2013', '1Q2013', '4Q2012', '3Q2012', '2Q2012', '1Q2012', '4Q2011', '3Q2011', '2Q2011', '1Q2011', '4Q2010', '3Q2010', '2Q2010', '1Q2010', '4Q2009', '3Q2009', '2Q2009', '1Q2009'
quarter = fundamental_data['Quarter']
fundamental_data.drop(labels=['Quarter'], axis=1, inplace=True)
fundamental_data.insert(0, 'Quarter', quarter)

fundamental_data.head()

,Quarter,Year,Month,Day,Ativo Total,Ativo Circulante,Caixa e Equivalentes de Caixa AC,Aplicações Financeiras AC,Contas a Receber AC,Estoques AC,Ativos Biológicos AC,Tributos a Recuperar AC,Despesas Antecipadas AC,Outros Ativos Circulantes AC,Ativo Realizavel Longo Prazo,Aplicações Financeiras Avaliadas a Valor Justo ACLP,Aplicações Financeiras Avaliadas ao Custo Amortizado ACLP,Contas a Receber ACLP,Estoques ACLP,Ativos Biológicos ACLP,Tributos Diferidos ACLP,Despesas Antecipadas ACLP,Créditos com Partes Relacionadas ACLP,Outros Ativos Não Circulantes ACLP,Investimentos,Imobilizado,Intangível,Diferido,Passivo Total,Passivo Circulante,Obrigações Sociais e Trabalhistas PC,Fornecedores PC,Obrigações Fiscais PC,Empréstimos e Financiamentos PC,Passivos com Partes Relacionadas PC,Dividendos e JCP a Pagar PC,Outros PC,Provisões PC,Passivos sobre Ativos Não-Correntes a Venda e Descontinuados PC,Passivo Não Circulante,Empréstimos e Financiamentos PNC,Passivos com Partes Relacionadas PNC,Outros PNC,Tributos Diferidos PNC,Adiantamento para Futuro Aumento Capital PNC,Provisões PNC,Passivos sobre Ativos Não-Correntes a Venda e Descontinuados PNC,Lucros e Receitas a Apropriar PNC,Participação dos Acionistas Não Controladores,Patrimônio Líquido,Capital Social Realizado,Reservas de Capital,Reservas de Reavaliação,Reservas de Lucros,Lucros/Prejuízos Acumulados,Ajustes de Avaliação Patrimonial,Ajustes Acumulados de Conversão,Outros Resultados Abrangentes,Adiantamento para Futuro Aumento Capital
0,2Q2019,2019,6,30,30.466.716,17.170.224,6.573.441,3.947.952,1.576.149,3.123.935,0,886.679,83.323,978.744,3.729.933,61.476,0,0,0,0,0,1.370.409,0,46.702,0,5.720.875,3.845.686,0,30.466.716,7.549.925,156.515,3.520.434,178.062,3.168.946,0,22.600,921,502.447,0,16.876.870,16.665.291,0,8.251,0,0,203.328,0,0,1.254.018,4.785.904,3.957.961,115.306,0,623.837,59.240,29.560,0,0,0
1,1Q2019,2019,3,31,32.632.261,19.639.284,7.725.360,4.616.139,1.695.657,3.795.014,0,808.076,79.648,919.390,3.701.606,41.417,0,0,0,0,0,1.361.352,0,51.464,0,5.495.443,3.795.926,0,32.632.261,8.874.362,145.691,4.698.474,161.449,3.230.820,0,126.215,1.229,510.484,0,17.794.312,17.577.052,0,8.160,0,0,209.101,0,0,1.301.020,4.662.565,3.957.961,102.809,0,623.837,-53.484,31.442,0,0,0
2,4Q2018,2018,12,31,28.304.656,17.513.619,6.813.846,3.295.849,1.870.081,3.506.678,0,906.836,59.036,1.061.295,3.380.094,0,0,0,0,0,0,1.197.780,0,51.008,0,3.647.720,3.763.221,0,28.304.656,8.402.748,152.235,4.973.577,325.947,2.290.854,0,126.215,1.534,532.386,0,13.615.783,13.389.983,0,8.039,0,0,217.761,0,0,1.361.528,4.924.597,3.957.961,90.311,0,845.540,0,30.785,0,0,0
3,3Q2018,2018,9,30,26.876.613,16.403.945,5.982.687,3.229.278,1.335.874,3.639.626,0,967.622,76.478,1.172.380,3.198.663,0,0,0,0,0,0,1.155.231,0,50.626,0,3.538.550,3.735.453,0,26.876.613,6.853.386,159.582,3.641.575,236.674,2.350.935,0,0,14.614,450.006,0,13.860.291,13.649.758,0,8.079,0,0,202.454,0,0,1.388.215,4.774.719,3.957.961,78.381,0,581.485,107.688,49.204,0,0,0
4,2Q2018,2018,6,30,26.015.085,15.810.592,5.201.874,4.158.143,1.126.535,3.207.755,0,843.403,73.311,1.199.571,3.052.373,35.840,0,0,0,0,0,1.104.241,0,47.900,0,3.429.345,3.722.775,0,26.015.085,6.475.659,145.617,2.814.037,208.490,2.933.797,0,0,19.566,354.152,0,13.427.922,13.195.491,0,8.364,0,0,224.067,0,0,1.402.201,4.709.303,3.926.518,95.002,0,597.146,46.294,44.343,0,0,0


Renomeei a coluna "Month" para "Cluster_Month", pois como se trata de trimestre, ele agrupa ali três meses correspondentes áquele trimestre, então sabemos que colunas onde tem Cluster_Month 3, englona na verdade, os meses 1,2 e 3. E assim por diante com os demais.

A coluna dia não nos é mais útil aqui, então tiramos a coluna.

In [6]:
fundamental_data.rename(columns={'Month' : 'Cluster_Month'},inplace = True)
fundamental_data.drop('Day', axis=1, inplace=True)

fundamental_data.head()

,Quarter,Year,Cluster_Month,Ativo Total,Ativo Circulante,Caixa e Equivalentes de Caixa AC,Aplicações Financeiras AC,Contas a Receber AC,Estoques AC,Ativos Biológicos AC,Tributos a Recuperar AC,Despesas Antecipadas AC,Outros Ativos Circulantes AC,Ativo Realizavel Longo Prazo,Aplicações Financeiras Avaliadas a Valor Justo ACLP,Aplicações Financeiras Avaliadas ao Custo Amortizado ACLP,Contas a Receber ACLP,Estoques ACLP,Ativos Biológicos ACLP,Tributos Diferidos ACLP,Despesas Antecipadas ACLP,Créditos com Partes Relacionadas ACLP,Outros Ativos Não Circulantes ACLP,Investimentos,Imobilizado,Intangível,Diferido,Passivo Total,Passivo Circulante,Obrigações Sociais e Trabalhistas PC,Fornecedores PC,Obrigações Fiscais PC,Empréstimos e Financiamentos PC,Passivos com Partes Relacionadas PC,Dividendos e JCP a Pagar PC,Outros PC,Provisões PC,Passivos sobre Ativos Não-Correntes a Venda e Descontinuados PC,Passivo Não Circulante,Empréstimos e Financiamentos PNC,Passivos com Partes Relacionadas PNC,Outros PNC,Tributos Diferidos PNC,Adiantamento para Futuro Aumento Capital PNC,Provisões PNC,Passivos sobre Ativos Não-Correntes a Venda e Descontinuados PNC,Lucros e Receitas a Apropriar PNC,Participação dos Acionistas Não Controladores,Patrimônio Líquido,Capital Social Realizado,Reservas de Capital,Reservas de Reavaliação,Reservas de Lucros,Lucros/Prejuízos Acumulados,Ajustes de Avaliação Patrimonial,Ajustes Acumulados de Conversão,Outros Resultados Abrangentes,Adiantamento para Futuro Aumento Capital
0,2Q2019,2019,6,30.466.716,17.170.224,6.573.441,3.947.952,1.576.149,3.123.935,0,886.679,83.323,978.744,3.729.933,61.476,0,0,0,0,0,1.370.409,0,46.702,0,5.720.875,3.845.686,0,30.466.716,7.549.925,156.515,3.520.434,178.062,3.168.946,0,22.600,921,502.447,0,16.876.870,16.665.291,0,8.251,0,0,203.328,0,0,1.254.018,4.785.904,3.957.961,115.306,0,623.837,59.240,29.560,0,0,0
1,1Q2019,2019,3,32.632.261,19.639.284,7.725.360,4.616.139,1.695.657,3.795.014,0,808.076,79.648,919.390,3.701.606,41.417,0,0,0,0,0,1.361.352,0,51.464,0,5.495.443,3.795.926,0,32.632.261,8.874.362,145.691,4.698.474,161.449,3.230.820,0,126.215,1.229,510.484,0,17.794.312,17.577.052,0,8.160,0,0,209.101,0,0,1.301.020,4.662.565,3.957.961,102.809,0,623.837,-53.484,31.442,0,0,0
2,4Q2018,2018,12,28.304.656,17.513.619,6.813.846,3.295.849,1.870.081,3.506.678,0,906.836,59.036,1.061.295,3.380.094,0,0,0,0,0,0,1.197.780,0,51.008,0,3.647.720,3.763.221,0,28.304.656,8.402.748,152.235,4.973.577,325.947,2.290.854,0,126.215,1.534,532.386,0,13.615.783,13.389.983,0,8.039,0,0,217.761,0,0,1.361.528,4.924.597,3.957.961,90.311,0,845.540,0,30.785,0,0,0
3,3Q2018,2018,9,26.876.613,16.403.945,5.982.687,3.229.278,1.335.874,3.639.626,0,967.622,76.478,1.172.380,3.198.663,0,0,0,0,0,0,1.155.231,0,50.626,0,3.538.550,3.735.453,0,26.876.613,6.853.386,159.582,3.641.575,236.674,2.350.935,0,0,14.614,450.006,0,13.860.291,13.649.758,0,8.079,0,0,202.454,0,0,1.388.215,4.774.719,3.957.961,78.381,0,581.485,107.688,49.204,0,0,0
4,2Q2018,2018,6,26.015.085,15.810.592,5.201.874,4.158.143,1.126.535,3.207.755,0,843.403,73.311,1.199.571,3.052.373,35.840,0,0,0,0,0,1.104.241,0,47.900,0,3.429.345,3.722.775,0,26.015.085,6.475.659,145.617,2.814.037,208.490,2.933.797,0,0,19.566,354.152,0,13.427.922,13.195.491,0,8.364,0,0,224.067,0,0,1.402.201,4.709.303,3.926.518,95.002,0,597.146,46.294,44.343,0,0,0


Correção de processamento, os dados vieram com "pontos" o que inviabiliza a manipulação númerica que o pandas compreende, assim tivemos que remover os pontos e atribuir o formato float, anteriormente veio com formato string.

Assim podemos ver a disposição dos números em relação a tabela gerada pela célula abaixo e a que tinhamos antes.

In [7]:
for key in ['Ativo Total','Ativo Circulante','Caixa e Equivalentes de Caixa AC','Aplicações Financeiras AC','Contas a Receber AC','Estoques AC', 'Ativos Biológicos AC','Tributos a Recuperar AC','Despesas Antecipadas AC','Outros Ativos Circulantes AC','Ativo Realizavel Longo Prazo ','Aplicações Financeiras Avaliadas a Valor Justo ACLP','Aplicações Financeiras Avaliadas ao Custo Amortizado ACLP','Contas a Receber ACLP','Estoques ACLP','Ativos Biológicos ACLP','Tributos Diferidos ACLP','Despesas Antecipadas ACLP','Créditos com Partes Relacionadas ACLP','Outros Ativos Não Circulantes ACLP','Investimentos','Imobilizado','Intangível','Diferido','Passivo Total','Passivo Circulante','Obrigações Sociais e Trabalhistas PC','Fornecedores PC','Obrigações Fiscais PC','Empréstimos e Financiamentos PC','Passivos com Partes Relacionadas PC','Dividendos e JCP a Pagar PC','Outros PC','Provisões PC', 'Passivos sobre Ativos Não-Correntes a Venda e Descontinuados PC', 'Passivo Não Circulante','Empréstimos e Financiamentos PNC','Passivos com Partes Relacionadas PNC','Outros PNC','Tributos Diferidos PNC','Adiantamento para Futuro Aumento Capital PNC','Provisões PNC','Passivos sobre Ativos Não-Correntes a Venda e Descontinuados PNC','Lucros e Receitas a Apropriar PNC','Participação dos Acionistas Não Controladores','Patrimônio Líquido','Capital Social Realizado','Reservas de Capital','Reservas de Reavaliação','Reservas de Lucros','Lucros/Prejuízos Acumulados','Ajustes de Avaliação Patrimonial','Ajustes Acumulados de Conversão','Outros Resultados Abrangentes','Adiantamento para Futuro Aumento Capital']:  
    fundamental_data[key] = fundamental_data[key].map(lambda x: str(x).replace(".", ""))
    fundamental_data[key] = fundamental_data[key].astype(float)


fundamental_data.head()

,Quarter,Year,Cluster_Month,Ativo Total,Ativo Circulante,Caixa e Equivalentes de Caixa AC,Aplicações Financeiras AC,Contas a Receber AC,Estoques AC,Ativos Biológicos AC,Tributos a Recuperar AC,Despesas Antecipadas AC,Outros Ativos Circulantes AC,Ativo Realizavel Longo Prazo,Aplicações Financeiras Avaliadas a Valor Justo ACLP,Aplicações Financeiras Avaliadas ao Custo Amortizado ACLP,Contas a Receber ACLP,Estoques ACLP,Ativos Biológicos ACLP,Tributos Diferidos ACLP,Despesas Antecipadas ACLP,Créditos com Partes Relacionadas ACLP,Outros Ativos Não Circulantes ACLP,Investimentos,Imobilizado,Intangível,Diferido,Passivo Total,Passivo Circulante,Obrigações Sociais e Trabalhistas PC,Fornecedores PC,Obrigações Fiscais PC,Empréstimos e Financiamentos PC,Passivos com Partes Relacionadas PC,Dividendos e JCP a Pagar PC,Outros PC,Provisões PC,Passivos sobre Ativos Não-Correntes a Venda e Descontinuados PC,Passivo Não Circulante,Empréstimos e Financiamentos PNC,Passivos com Partes Relacionadas PNC,Outros PNC,Tributos Diferidos PNC,Adiantamento para Futuro Aumento Capital PNC,Provisões PNC,Passivos sobre Ativos Não-Correntes a Venda e Descontinuados PNC,Lucros e Receitas a Apropriar PNC,Participação dos Acionistas Não Controladores,Patrimônio Líquido,Capital Social Realizado,Reservas de Capital,Reservas de Reavaliação,Reservas de Lucros,Lucros/Prejuízos Acumulados,Ajustes de Avaliação Patrimonial,Ajustes Acumulados de Conversão,Outros Resultados Abrangentes,Adiantamento para Futuro Aumento Capital
0,2Q2019,2019,6,30466716.0,17170224.0,6573441.0,3947952.0,1576149.0,3123935.0,0.0,886679.0,83323.0,978744.0,3729933.0,61476.0,0.0,0.0,0.0,0.0,0.0,1370409.0,0.0,46702.0,0.0,5720875.0,3845686.0,0.0,30466716.0,7549925.0,156515.0,3520434.0,178062.0,3168946.0,0.0,22600.0,921.0,502447.0,0.0,16876870.0,16665291.0,0.0,8251.0,0.0,0.0,203328.0,0.0,0.0,1254018.0,4785904.0,3957961.0,115306.0,0.0,623837.0,59240.0,29560.0,0.0,0.0,0.0
1,1Q2019,2019,3,32632261.0,19639284.0,7725360.0,4616139.0,1695657.0,3795014.0,0.0,808076.0,79648.0,919390.0,3701606.0,41417.0,0.0,0.0,0.0,0.0,0.0,1361352.0,0.0,51464.0,0.0,5495443.0,3795926.0,0.0,32632261.0,8874362.0,145691.0,4698474.0,161449.0,3230820.0,0.0,126215.0,1229.0,510484.0,0.0,17794312.0,17577052.0,0.0,8160.0,0.0,0.0,209101.0,0.0,0.0,1301020.0,4662565.0,3957961.0,102809.0,0.0,623837.0,-53484.0,31442.0,0.0,0.0,0.0
2,4Q2018,2018,12,28304656.0,17513619.0,6813846.0,3295849.0,1870081.0,3506678.0,0.0,906836.0,59036.0,1061295.0,3380094.0,0.0,0.0,0.0,0.0,0.0,0.0,1197780.0,0.0,51008.0,0.0,3647720.0,3763221.0,0.0,28304656.0,8402748.0,152235.0,4973577.0,325947.0,2290854.0,0.0,126215.0,1534.0,532386.0,0.0,13615783.0,13389983.0,0.0,8039.0,0.0,0.0,217761.0,0.0,0.0,1361528.0,4924597.0,3957961.0,90311.0,0.0,845540.0,0.0,30785.0,0.0,0.0,0.0
3,3Q2018,2018,9,26876613.0,16403945.0,5982687.0,3229278.0,1335874.0,3639626.0,0.0,967622.0,76478.0,1172380.0,3198663.0,0.0,0.0,0.0,0.0,0.0,0.0,1155231.0,0.0,50626.0,0.0,3538550.0,3735453.0,0.0,26876613.0,6853386.0,159582.0,3641575.0,236674.0,2350935.0,0.0,0.0,14614.0,450006.0,0.0,13860291.0,13649758.0,0.0,8079.0,0.0,0.0,202454.0,0.0,0.0,1388215.0,4774719.0,3957961.0,78381.0,0.0,581485.0,107688.0,49204.0,0.0,0.0,0.0
4,2Q2018,2018,6,26015085.0,15810592.0,5201874.0,4158143.0,1126535.0,3207755.0,0.0,843403.0,73311.0,1199571.0,3052373.0,35840.0,0.0,0.0,0.0,0.0,0.0,1104241.0,0.0,47900.0,0.0,3429345.0,3722775.0,0.0,26015085.0,6475659.0,145617.0,2814037.0,208490.0,2933797.0,0.0,0.0,19566.0,354152.0,0.0,13427922.0,13195491.0,0.0,8364.0,0.0,0.0,224067.0,0.0,0.0,1402201.0,4709303.0,3926518.0,95002.0,0.0,597146.0,46294.0,44343.0,0.0,0.0,0.0


# DADOS DE PREÇO - API YAHOO FINANCE

Após determinandos os dados fundamentalistas da ação, chegou a hora de extrairmos os dados de preço da API aberta da Yahoo Finance.

Lembrando que temos que convertê-las ao formato de escala tempora de trimestre, para que possamos concatêna-las aos dados fundamentalistas, formando um dataset único da ação. Dados de preço e dados fundamentalistas referentes ao mesmo espaço temporal para cada amostra.

Abaixo, primeiro atribuimos a data de inicio e de fim que coletaremos os dados de preço, e depois trazemos o dataset com base nas datas, e 'm' nos diz respeito a escala temporal mês, como não há trimestre, tivemos que fazer a seguir a manipulação no código, com base nos dados originalmente mensais vindo da API.

In [8]:
####################### COMEÇA O CÓDIGO QUE EXTRAI DADOS DE PREÇO ###############################

start = datetime(2008, 12, 31)
end = datetime(2019, 5, 31)
price_data = web.get_data_yahoo('LAME4.SA', start, end, interval = 'm')

price_data.head()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2009-01-01,3.44193,2.92182,3.18697,3.25836,91625694.0,2.972662
2009-02-01,3.22777,2.74845,3.22777,2.88102,79620968.0,2.628408
2009-03-01,3.69689,2.69746,2.83003,3.30935,131043759.0,3.019181
2009-04-01,4.65553,3.19207,3.21757,4.58924,127634375.0,4.207708
2009-05-01,5.22154,4.33429,4.68103,4.80341,127728703.0,4.404072


As datas na coluna "Date" estavam como índíce, e usamos o code abaixo para tornar "Date" uma coluna normal do dataset, já que vamos precisar manipula-la.

In [9]:
price_data["Date"] = price_data.index
price_data.reset_index(drop=True, inplace=True)


date = price_data['Date']
price_data.drop(labels=['Date'], axis=1,inplace = True)
price_data.insert(0, 'Date', date)

price_data.head()

,Date,High,Low,Open,Close,Volume,Adj Close
0,2009-01-01,3.44193,2.92182,3.18697,3.25836,91625694.0,2.972662
1,2009-02-01,3.22777,2.74845,3.22777,2.88102,79620968.0,2.628408
2,2009-03-01,3.69689,2.69746,2.83003,3.30935,131043759.0,3.019181
3,2009-04-01,4.65553,3.19207,3.21757,4.58924,127634375.0,4.207708
4,2009-05-01,5.22154,4.33429,4.68103,4.80341,127728703.0,4.404072


Aqui começa a manipulação para transformarmos os dados de preço de escala mensal para escala trimestral.

Tal qual nos dados fundamentalistas, da coluna Date, criamos as colunas referentes a "dia", "mês" e "ano". Assim a coluna Date não nos é mais útil.

Manipulamos para colocar as três colunas referentes a data nas três primeiras posições das colunas no dataset.

In [10]:
price_data["Day"] = price_data['Date'].map(lambda x: x.day)
price_data["Month"] = price_data['Date'].map(lambda x: x.month)
price_data["Year"] = price_data['Date'].map(lambda x: x.year)

price_data.drop(columns = 'Date', inplace = True)



year = price_data['Year']
price_data.drop(labels=['Year'], axis=1,inplace = True)
price_data.insert(0, 'Year', year)

month = price_data['Month']
price_data.drop(labels=['Month'], axis=1,inplace = True)
price_data.insert(1, 'Month', month)

day = price_data['Day']
price_data.drop(labels=['Day'], axis=1,inplace = True)
price_data.insert(2, 'Day', day)

price_data.head()

,Year,Month,Day,High,Low,Open,Close,Volume,Adj Close
0,2009,1,1,3.44193,2.92182,3.18697,3.25836,91625694.0,2.972662
1,2009,2,1,3.22777,2.74845,3.22777,2.88102,79620968.0,2.628408
2,2009,3,1,3.69689,2.69746,2.83003,3.30935,131043759.0,3.019181
3,2009,4,1,4.65553,3.19207,3.21757,4.58924,127634375.0,4.207708
4,2009,5,1,5.22154,4.33429,4.68103,4.80341,127728703.0,4.404072


Nessa parte, o "for" percorre todas as linhas do dataset, e tal qual no padrão dos dados fundamentalistas, temos que atribuir aquele padrão que chamei de "Cluster_Month", onde o mês 3 representa o trimestre dos meses 1,2 e 3. Mês 6 o trimestre dos meses 4,5 e 6. E assim por diante.

Após, deletamos a coluna "dia", que não nos é útil.

In [11]:
for i,row in price_data.iterrows():   
    if (row['Month'] == 1) | (row['Month'] == 2) | (row['Month'] == 3):
        price_data.loc[i,'Month'] = 3
    
    elif (row['Month'] == 4) | (row['Month'] == 5) | (row['Month'] == 6):
         price_data.loc[i,'Month'] = 6
        
    elif (row['Month'] == 7) | (row['Month'] == 8) | (row['Month'] == 9):
         price_data.loc[i,'Month'] = 9
        
    else: 
          price_data.loc[i,'Month'] = 12



price_data.drop('Day', axis=1, inplace=True)

price_data.head()

,Year,Month,High,Low,Open,Close,Volume,Adj Close
0,2009,3,3.44193,2.92182,3.18697,3.25836,91625694.0,2.972662
1,2009,3,3.22777,2.74845,3.22777,2.88102,79620968.0,2.628408
2,2009,3,3.69689,2.69746,2.83003,3.30935,131043759.0,3.019181
3,2009,6,4.65553,3.19207,3.21757,4.58924,127634375.0,4.207708
4,2009,6,5.22154,4.33429,4.68103,4.80341,127728703.0,4.404072


E aqui tempos uma parte bem interessante. Agora agrupamos cada uma das três linhas correspondentes ao trimestre do ano correspondente.

Podemos olhar juntos para a primeira instrução abaixo e para a tabela acima para entender. Agrupamos por Ano e Mês, assim podemos ver as três primeiras linhas com "Month" igual a 3(na verdade são 1,2 e 3) que foram manipulados para facilitar esse processo. Então iremos agrupar essas três linhas, transformando em uma linha só correspondente ao primeiro trimestre de 2014 nesse caso. E assim convertemos os parâmetros de preço conforme suas especificidades.

- Abertura(Open): Utilização do método “first”, atribuindo o primeiro valor do trimestre.
   
- Fechamento(Close): Utilização do método “last”, atribuindo o último valor do trimestre.
      
- Máximo(High):Utilização do método “max”, atribuindo o máximo valor do trimestre.
 
- Mínimo(Low):Utilização do método “min”, atribuindo o mínimo valor do trimestre.

- Volume(Volume): Utilização do método “mean”, atribuindo a média dos valores do trimestre.

- Fechamento Ajustado(Adj Close):Utilização do método “mean”, atribuindo a média dos valores do trimestre.



Depois, novamente o processo de criar uma coluna "Quartes" com uma identificação mais intuitiva do trimestre e ano, e coloca-lo como a primeira posição das colunas.

Além de renomear a coluna "Month" para "Cluster_Month", tudo no mesmo padrão dos dados fundamentalistas.

In [12]:
final_data_price = price_data.groupby(['Year','Month']).agg({'High': 'max', 'Low': 'min', 'Open':'first', 'Close' : 'last', 'Volume':'mean', 'Adj Close':'mean'}).reset_index()


final_data_price['Quarter'] = '1Q2009', '2Q2009', '3Q2009', '4Q2009', '1Q2010', '2Q2010', '3Q2010', '4Q2010', '1Q2011', '2Q2011', '3Q2011', '4Q2011', '1Q2012', '2Q2012', '3Q2012','4Q2012', '1Q2013', '2Q2013', '3Q2013', '4Q2013', '1Q2014', '2Q2014', '3Q2014', '4Q2014', '1Q2015', '2Q2015', '3Q2015', '4Q2015', '1Q2016', '2Q2016', '3Q2016', '4Q2016', '1Q2017', '2Q2017', '3Q2017', '4Q2017', '1Q2018', '2Q2018', '3Q2018', '4Q2018','1Q2019','2Q2019'

quarter = final_data_price['Quarter']
final_data_price.drop(labels=['Quarter'], axis=1, inplace=True)
final_data_price.insert(0, 'Quarter', quarter)

final_data_price.rename(columns={'Month' : 'Cluster_Month'},inplace = True)

final_data_price.head()

,Quarter,Year,Cluster_Month,High,Low,Open,Close,Volume,Adj Close
0,1Q2009,2009,3,3.69689,2.69746,3.18697,3.30935,1.007635e+08,2.873417
1,2Q2009,2009,6,5.22154,3.19207,3.21757,4.68103,1.156743e+08,4.301215
2,3Q2009,2009,9,6.10369,4.50766,4.71672,6.09859,1.024660e+08,5.253402
3,4Q2009,2009,12,8.21475,5.44080,6.13429,7.91899,1.126373e+08,6.487664
4,1Q2010,2010,3,8.10256,6.24647,8.00058,6.75639,9.978678e+07,6.152610


# UNINDO DADOS DE PREÇO E DADOS FUNDAMENTALISTAS

Simplesmente fazendo um merge entre os dados de preço e fundamentalistas que tratamos até aqui.

A coluna "Year" e "Month" ficará repetida, então precisaremos renomear cada uma e excluir as outras duas que são a mesma coisa.

In [13]:
# MESCLA DOS DADOS DE PREÇO E OS FUNDAMENTALISTAS

dataset = pd.merge(final_data_price, fundamental_data, how='left', on='Quarter')

dataset.rename(columns={'Year_x' : 'Year'},inplace = True)
dataset.rename(columns={'Cluster_Month_x' : 'Cluster_Month'},inplace = True)

dataset.drop('Year_y', axis=1, inplace=True)

dataset.drop('Cluster_Month_y', axis=1, inplace=True)

dataset.head()

,Quarter,Year,Cluster_Month,High,Low,Open,Close,Volume,Adj Close,Ativo Total,Ativo Circulante,Caixa e Equivalentes de Caixa AC,Aplicações Financeiras AC,Contas a Receber AC,Estoques AC,Ativos Biológicos AC,Tributos a Recuperar AC,Despesas Antecipadas AC,Outros Ativos Circulantes AC,Ativo Realizavel Longo Prazo,Aplicações Financeiras Avaliadas a Valor Justo ACLP,Aplicações Financeiras Avaliadas ao Custo Amortizado ACLP,Contas a Receber ACLP,Estoques ACLP,Ativos Biológicos ACLP,Tributos Diferidos ACLP,Despesas Antecipadas ACLP,Créditos com Partes Relacionadas ACLP,Outros Ativos Não Circulantes ACLP,Investimentos,Imobilizado,Intangível,Diferido,Passivo Total,Passivo Circulante,Obrigações Sociais e Trabalhistas PC,Fornecedores PC,Obrigações Fiscais PC,Empréstimos e Financiamentos PC,Passivos com Partes Relacionadas PC,Dividendos e JCP a Pagar PC,Outros PC,Provisões PC,Passivos sobre Ativos Não-Correntes a Venda e Descontinuados PC,Passivo Não Circulante,Empréstimos e Financiamentos PNC,Passivos com Partes Relacionadas PNC,Outros PNC,Tributos Diferidos PNC,Adiantamento para Futuro Aumento Capital PNC,Provisões PNC,Passivos sobre Ativos Não-Correntes a Venda e Descontinuados PNC,Lucros e Receitas a Apropriar PNC,Participação dos Acionistas Não Controladores,Patrimônio Líquido,Capital Social Realizado,Reservas de Capital,Reservas de Reavaliação,Reservas de Lucros,Lucros/Prejuízos Acumulados,Ajustes de Avaliação Patrimonial,Ajustes Acumulados de Conversão,Outros Resultados Abrangentes,Adiantamento para Futuro Aumento Capital
0,1Q2009,2009,3,3.69689,2.69746,3.18697,3.30935,1.007635e+08,2.873417,6011012.0,4253609.0,1759009.0,0.0,668783.0,1127896.0,0.0,0.0,0.0,697921.0,232936.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,82400.0,150536.0,0.0,501336.0,853998.0,169133.0,6011012.0,3484416.0,0.0,1331885.0,68045.0,1862446.0,0.0,42540.0,131381.0,48119.0,0.0,2099492.0,1931606.0,0.0,38288.0,0.0,0.0,129598.0,0.0,0.0,100440.0,326664.0,118365.0,3703.0,0.0,201930.0,2527.0,139.0,0.0,0.0,0.0
1,2Q2009,2009,6,5.22154,3.19207,3.21757,4.68103,1.156743e+08,4.301215,5889976.0,4152469.0,1502144.0,0.0,901842.0,1066206.0,0.0,0.0,0.0,682277.0,213588.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,50196.0,163392.0,0.0,486889.0,879076.0,157954.0,5889976.0,3152749.0,0.0,1366808.0,96467.0,1490615.0,0.0,404.0,146526.0,51929.0,0.0,2332889.0,2163195.0,0.0,36994.0,0.0,0.0,132700.0,0.0,0.0,106384.0,297954.0,85261.0,3932.0,0.0,201930.0,6763.0,68.0,0.0,0.0,0.0
2,3Q2009,2009,9,6.10369,4.50766,4.71672,6.09859,1.024660e+08,5.253402,5903648.0,4189572.0,1432246.0,0.0,977318.0,1222899.0,0.0,0.0,0.0,557109.0,206261.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,50657.0,155604.0,0.0,473308.0,887702.0,146805.0,5903648.0,2736892.0,0.0,1326331.0,115192.0,1091211.0,0.0,319.0,150272.0,53567.0,0.0,2781487.0,2611263.0,0.0,46695.0,0.0,0.0,123529.0,0.0,0.0,106163.0,279106.0,85177.0,4113.0,0.0,174869.0,15090.0,-143.0,0.0,0.0,0.0
3,4Q2009,2009,12,8.21475,5.44080,6.13429,7.91899,1.126373e+08,6.487664,6747984.0,4927916.0,2208863.0,0.0,999734.0,1263404.0,0.0,0.0,0.0,455915.0,272063.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,35735.0,236328.0,0.0,494536.0,917923.0,135546.0,6747984.0,3115660.0,0.0,1701718.0,161936.0,899189.0,0.0,51121.0,251639.0,50057.0,0.0,3145069.0,2954700.0,0.0,37028.0,0.0,0.0,153341.0,0.0,0.0,107139.0,380116.0,85052.0,5303.0,0.0,288465.0,0.0,1296.0,0.0,0.0,0.0
4,1Q2010,2010,3,8.10256,6.24647,8.00058,6.75639,9.978678e+07,6.152610,6221174.0,4406166.0,1383439.0,0.0,1316609.0,1207167.0,0.0,0.0,0.0,498951.0,266441.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,35273.0,231168.0,0.0,485848.0,937693.0,125026.0,6221174.0,2775697.0,0.0,1331884.0,105230.0,1033676.0,0.0,48623.0,206300.0,49984.0,0.0,2926169.0,2733769.0,0.0,37502.0,0.0,0.0,154898.0,0.0,0.0,108740.0,410568.0,84960.0,5903.0,0.0,288465.0,30881.0,359.0,0.0,0.0,0.0


Ao analisar o dataset, decidi excluir colunas que tinha mais de 1/3 dos dados faltantes, seguindo a lista abaixo de colunas "dropadas".

In [14]:
# exclui colunas que tem mais de 1/3 dos dados como zero ou faltante
dataset = dataset.drop('Ativos Biológicos AC',axis=1)
dataset = dataset.drop('Despesas Antecipadas AC',axis=1)
dataset = dataset.drop('Aplicações Financeiras Avaliadas a Valor Justo ACLP',axis=1)
dataset = dataset.drop('Aplicações Financeiras Avaliadas ao Custo Amortizado ACLP',axis=1)
dataset = dataset.drop('Estoques ACLP',axis=1)
dataset = dataset.drop('Ativos Biológicos ACLP',axis=1)
dataset = dataset.drop('Despesas Antecipadas ACLP',axis=1)
dataset = dataset.drop('Créditos com Partes Relacionadas ACLP',axis=1)
dataset = dataset.drop('Diferido',axis=1)
dataset = dataset.drop('Passivos com Partes Relacionadas PC',axis=1)
dataset = dataset.drop('Dividendos e JCP a Pagar PC',axis=1)
dataset = dataset.drop('Provisões PC',axis=1)
dataset = dataset.drop('Passivos sobre Ativos Não-Correntes a Venda e Descontinuados PC',axis=1)
dataset = dataset.drop('Passivos com Partes Relacionadas PNC',axis=1)
dataset = dataset.drop('Adiantamento para Futuro Aumento Capital PNC',axis=1)
dataset = dataset.drop('Passivos sobre Ativos Não-Correntes a Venda e Descontinuados PNC',axis=1)
dataset = dataset.drop('Lucros e Receitas a Apropriar PNC',axis=1)
dataset = dataset.drop('Reservas de Reavaliação',axis=1)
dataset = dataset.drop('Ajustes de Avaliação Patrimonial',axis=1)
dataset = dataset.drop('Ajustes Acumulados de Conversão',axis=1)
dataset = dataset.drop('Outros Resultados Abrangentes',axis=1)
dataset = dataset.drop('Adiantamento para Futuro Aumento Capital',axis=1)

# APLICAÇÃO DO ALGORTIMO KNN PARA PREENCHIMENTO DOS DADOS FALTANTES

Abaixo, separei apenas as colunas que serão imputadas para treinamento, tirando as de data e os id's. Assim ao invés de um método tradicional como preencher dados faltantes com média ou mediana, utilizei a metodologia com a biblioteca fancy impute, para aplicar KNN e preencher os dados faltantes com esse algortimo.

Na célula abaixo a separação.

Após trocar os valores 0 para "NaN", então realizamos o preenchimento de fato com o KNN.

In [15]:
#separa as colunas que serão usadas para treinamento em 'dataset'
pricing = dataset.iloc[0:,0:3]
dataset = dataset.iloc[0:,3:]
dataset.to_numpy()

X = dataset.replace(0, np.nan, inplace=True)
dataset.replace(0, np.nan, inplace=True)
final_data = KNN(k=3).fit_transform(dataset)

# ela tira as labels das colunas e aqui as coloco de volta
dataset = pd.DataFrame(data = final_data, columns = ['High','Low','Open','Close','Volume','Adj Close','Ativo Total','Ativo Circulante','Caixa e Equivalentes de Caixa AC','Aplicações Financeiras AC','Contas a Receber AC','Estoques AC','Tributos a Recuperar AC','Outros Ativos Circulantes AC','Ativo Realizavel Longo Prazo','Contas a Receber ACLP','Tributos Diferidos ACLP','Outros Ativos Não Circulantes ACLP','Investimentos','Imobilizado','Intangível','Passivo Total','Passivo Circulante','Obrigações Sociais e Trabalhistas PC','Fornecedores PC','Obrigações Fiscais PC','Empréstimos e Financiamentos PC','Outros PC','Passivo Não Circulante','Empréstimos e Financiamentos PNC','Outros PNC','Tributos Diferidos PNC','Provisões PNC','Participação dos Acionistas Não Controladores','Patrimônio Líquido','Capital Social Realizado','Reservas de Capital','Reservas de Lucros','Lucros/Prejuízos Acumulados'])
dataset = pd.concat([pricing, dataset], axis=1)

inference = dataset.iloc[-1,0:].to_frame().T
inference

Imputing row 1/42 with 7 missing, elapsed time: 0.002
[KNN] Warning: 84/1638 still missing after imputation, replacing with 0


,Quarter,Year,Cluster_Month,High,Low,Open,Close,Volume,Adj Close,Ativo Total,Ativo Circulante,Caixa e Equivalentes de Caixa AC,Aplicações Financeiras AC,Contas a Receber AC,Estoques AC,Tributos a Recuperar AC,Outros Ativos Circulantes AC,Ativo Realizavel Longo Prazo,Contas a Receber ACLP,Tributos Diferidos ACLP,Outros Ativos Não Circulantes ACLP,Investimentos,Imobilizado,Intangível,Passivo Total,Passivo Circulante,Obrigações Sociais e Trabalhistas PC,Fornecedores PC,Obrigações Fiscais PC,Empréstimos e Financiamentos PC,Outros PC,Passivo Não Circulante,Empréstimos e Financiamentos PNC,Outros PNC,Tributos Diferidos PNC,Provisões PNC,Participação dos Acionistas Não Controladores,Patrimônio Líquido,Capital Social Realizado,Reservas de Capital,Reservas de Lucros,Lucros/Prejuízos Acumulados
41,2Q2019,2019,6,17.1313,14.5302,16.8324,16.0849,1.36828e+08,15.6458,3.04667e+07,1.71702e+07,6.57344e+06,3.94795e+06,1.57615e+06,3.12394e+06,886679,978744,3.72993e+06,0,1.03246e+06,46702,0,5.72088e+06,3.84569e+06,3.04667e+07,7.54992e+06,156515,3.52043e+06,178062,3.16895e+06,921,1.68769e+07,1.66653e+07,8251,32114,203328,1.25402e+06,4.7859e+06,3.95796e+06,115306,623837,59240


In [16]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 42 entries, 0 to 41
Data columns (total 42 columns):
Quarter                                          42 non-null object
Year                                             42 non-null int64
Cluster_Month                                    42 non-null int64
High                                             42 non-null float64
Low                                              42 non-null float64
Open                                             42 non-null float64
Close                                            42 non-null float64
Volume                                           42 non-null float64
Adj Close                                        42 non-null float64
Ativo Total                                      42 non-null float64
Ativo Circulante                                 42 non-null float64
Caixa e Equivalentes de Caixa AC                 42 non-null float64
Aplicações Financeiras AC                        42 non-null float64
Conta

# AQUI FINALMENTE PRODUZIMO A NOSSA AMOSTRA QUE SERÁ IMPUTADA AO NOSSO MODELO, SIMULANDO UMA SITUAÇÃO REAL. AO IMPUTAR OS DADOS DE PREÇO E DE FUNDAMENTOS CONFORME OS USADOS NO MODELO, QUAL STATUS DO TARGET ELE RETORNARÁ FRENTE A UMA POSSIVEL VALORIZAÇÃO ?

In [17]:
inference = inference.iloc[0:,3:]
inference

,High,Low,Open,Close,Volume,Adj Close,Ativo Total,Ativo Circulante,Caixa e Equivalentes de Caixa AC,Aplicações Financeiras AC,Contas a Receber AC,Estoques AC,Tributos a Recuperar AC,Outros Ativos Circulantes AC,Ativo Realizavel Longo Prazo,Contas a Receber ACLP,Tributos Diferidos ACLP,Outros Ativos Não Circulantes ACLP,Investimentos,Imobilizado,Intangível,Passivo Total,Passivo Circulante,Obrigações Sociais e Trabalhistas PC,Fornecedores PC,Obrigações Fiscais PC,Empréstimos e Financiamentos PC,Outros PC,Passivo Não Circulante,Empréstimos e Financiamentos PNC,Outros PNC,Tributos Diferidos PNC,Provisões PNC,Participação dos Acionistas Não Controladores,Patrimônio Líquido,Capital Social Realizado,Reservas de Capital,Reservas de Lucros,Lucros/Prejuízos Acumulados
41,17.1313,14.5302,16.8324,16.0849,1.36828e+08,15.6458,3.04667e+07,1.71702e+07,6.57344e+06,3.94795e+06,1.57615e+06,3.12394e+06,886679,978744,3.72993e+06,0,1.03246e+06,46702,0,5.72088e+06,3.84569e+06,3.04667e+07,7.54992e+06,156515,3.52043e+06,178062,3.16895e+06,921,1.68769e+07,1.66653e+07,8251,32114,203328,1.25402e+06,4.7859e+06,3.95796e+06,115306,623837,59240


In [18]:
inference.to_csv("ROW_INFERENCE_LAME4.csv", index= False)